# Проект спринта 7. Анализ данных для команды игры "Секреты Темнолесья


### Цели и задачи проекта

Познакомиться с данными датасета `/datasets/new_games.csv`,который содержит исторические данные об играх. Необходимо проверить их корректность и провести предобработку, получив необходимый срез данных.

### Описание данных

* `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка данных и знакомство с ними 
2. Проверка ошибок данных и их предобработка
 1. Названия, или метки, столбцов датафрейма
 2. Типы данных
 3. Наличие пропусков в данных
 4. Явные и неявные дубликаты в данных
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод

## 1. Загрузка данных и знакомство с ними

Загружаем библиотеку Pandas и датасет `/datasets/new_games.csv`.


In [1]:
# Эта ячейка исправит проблему SSL если она есть
import ssl
try:
    # Пробуем нормальный режим
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
import pandas as pd

In [3]:
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

Знакомимся с данными: выводим результат метода `info()` и первые строки `.
<a id='1'></a>

In [4]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [5]:
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Выводы о полученных данных:
 - Всего в датафрейме 16956 строк и 11 столбцов.
 - Есть пропуски в таких столбцах: `Name`; `Year of Releaese`; `Genre`; `Critic Score`; `User Score`; `Rating`.
 - Соотвественно, столбцы без пропусков: `Platform`; `NA sales`; `EU sales`; `JP sales`; `Other sales`.
 - Есть столбцы с типом данных `object`, `float64`, нет столбцов с типом данных `integer` и `bool`. Также можно заметить, что столбцы `NA sales`; `EU sales`; `JP sales`; `Other sales`, описывающие одинаковую информацию в одинаковых единицах измерения имеют разные типы данных (`NA sales`, `Other sales`- `float64`, `EU sales`; `JP sales` - `object`). В будущем это придется исправить. 
 - `User Score` оптимально будет привести к типу данных `float64`.
 - `Year of Releaese` оптимально будет привести к типу данных `int64`.

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Ещё раз выводим на экран названия всех столбцов датафрейма через `.info()` и проверяем их стиль написания.

In [6]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


- Можно заметить, что названия столбцов не приведены к стилю snake case. Заменим все пробелы на _ , приведем названия к нижнему регистру. <a id='2A'></a>

In [7]:
games.columns = games.columns.str.replace(' ', '_')

In [8]:
games.columns = games.columns.str.lower()
display(games)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.0,0.01,0.00,NaN,NaN,NaN
16952,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.0,0.00,NaN,NaN,NaN
16953,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.0,0.01,0.00,NaN,NaN,NaN
16954,Spirits & Spells,GBA,2003.0,Platform,0.01,0.0,0.0,0.00,NaN,NaN,NaN


- Заменили все пропуски в числовых колонках на `NaN`. Также для удобства преобразовали колонку `year_of_release` в формат `Int64`. <a id='2B'></a>

In [9]:
games['critic_score'] = pd.to_numeric(games['critic_score'], errors = 'coerce')
games['user_score'] = pd.to_numeric(games['user_score'], errors = 'coerce')
games['year_of_release'] = pd.to_datetime(games['year_of_release'], format= '%Y', errors='coerce')
games['year_of_release'] = games['year_of_release'].dt.year.astype('Int64')
games['eu_sales'] = pd.to_numeric(games['eu_sales'], errors = 'coerce')
games['jp_sales'] = pd.to_numeric(games['jp_sales'], errors = 'coerce')
display(games, games.dtypes)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16952,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16953,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16954,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


name                object
platform            object
year_of_release      Int64
genre               object
na_sales           float64
eu_sales           float64
jp_sales           float64
other_sales        float64
critic_score       float64
user_score         float64
rating              object
dtype: object

### 2.3. Наличие пропусков в данных

- Смотрим в каких столбцах встречаются пропуски. 
<a id='2C'></a>

In [10]:
games.isna()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,True,True
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
16951,False,False,False,False,False,False,False,False,True,True,True
16952,False,False,False,False,False,False,False,False,True,True,True
16953,False,False,False,False,False,False,False,False,True,True,True
16954,False,False,False,False,False,False,False,False,True,True,True


- Рассчитываем количество пропусков в строках в каждом столбце.

In [11]:
games.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

- Рассчитаем долю пропусков от общего количества строк (в процентах).

In [12]:
(games.isna().sum() / len(games))*100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

- Есть пропуски в таких столбцах: `name`; `year_of_release`; `genre`; `critic_score`; `user_score`; `rating`. После преобразоаний типов данных обнаружилось, что в строках `eu_sales ` и `jp_sales` тоже есть пропуски. Это произошло из-за приведения данныхот тип `object` к `float64`.
- Строки с пропусками в колонке `name` относятся к типу MCAR и оставляют всего около 0.01%. Скорее всего их придется удалить для более эффективного анализа остальных данных. Тем более такое неболшое количество пропущенных данных вряд ли существенно исказит итоговый результат. 
- Строки с пропусками в колонке `genre` относятся к типу MCAR и оставляют всего около 0.01%. Чтобы не потерять данные о существующих играх, скорее всего придется заменить пропуски в данных строках.
- Строки с пропусками в колонке `year_of_release` относятся к типу MCAR и оставляют около 1,6%. У каждой игры есть год выпуска, но по той или иной причине он пропущен. Причины этого сложно предположить, поэтому отнесем этот пропуск к виду совершенно случайных. 
- Строки с пропусками в колонке `eu_sales ` и `jp_sales` составили 0.035% и 0.02%. Такие пропуски вполне допустимы, ведь игры могли не продаваться на территории Европы и Японии. Скорее всего эти пропуски придется не учитывать, либо заменить средними значениями по столбцу. В целом, такие пропуски можно даже удалить, так как они составляют менее 1%.
- Строки с пропусками в колонках `rating` оставляют около 40%. Скорее всего люди просто не оценивали некоторые игры или разработчики не давали им такой возможности выставить оценку. 
- Строки с пропусками в колонках `user_score` состаялют 54,6%. Это самая большая доля пропусков в датафрейме, ее нельзя игнорировать, придется заменять. Организация, которая присваивает рейтиг ESRB могла не оценить игры, так как сами разработчики не решили вопрос с подачей заявки на оценку или игра не популярна и организация просто не обратила внимания на эту игру. Причины зависят от того, каким образом игры попадают на оценку в организацию. Такие пропуски точно придется заполнить, чтобы не упустить огромный массив данных для анализа.
- Строки с пропусками в колонке `critic_score` составляют около 51%. Скорее всего критики просто не оценивают непопулярные игры, но это тоже стоит проверить, обратив внимание на количество продаж таких игр с пропусками в рейтингах. 


In [13]:
# Удаляем значения из колонки names, так как пропуски составляют <1%
games = games.dropna(subset = ['name']) 

# Удаляем значения из колонок eu_sales и jp_sales, так как пропуски составляют <1%
games = games.dropna(subset = ['eu_sales']) 
games = games.dropna(subset = ['jp_sales']) 

# Заполняем пропуски значением -1,так как по условию оценки в колонке critic_score начинаются с 0
games['critic_score'] = games['critic_score'].fillna(-1) 

# Заполняем пропуски значением -1,так как по условю оценки в колонке user_score начинаются с 0
games['user_score'] = games['user_score'].fillna(-1) 

# Заполняем пропуски фразой "без рейтинга", так как в колонке rating представлены буквы и, соответственно, 
# колонка имеет тип object
# Цифры в таком случае будут выглядеть странно, создавать путаницу 
games['rating'] = games ['rating'].fillna('без рейтинга')


# Заполняем пропуски значением 0,так как в колонке year_of_release начинаются представлены даты. 
games['year_of_release'] = games['year_of_release'].fillna(0) 

display(games.isna().sum())

name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

### 2.4. Явные и неявные дубликаты в данных

- Найдем количество уникальных строк и неявные дубликаты. <a id='2D'></a>

In [14]:
nuniqe_name = games.nunique()
display(nuniqe_name)

display()

unique_genre = games['genre'].unique()
display(unique_genre)

display()

unique_year_of_release = games['year_of_release'].unique()
display(unique_year_of_release)

display()

unique_platform = games['platform'].unique()
display(unique_platform)

display()

unique_rating = games['rating'].unique()
display(unique_rating)

name               11556
platform              31
year_of_release       38
genre                 24
na_sales             402
eu_sales             307
jp_sales             244
other_sales          155
critic_score          83
user_score            96
rating                 9
dtype: int64

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

<IntegerArray>
[2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010, 2013, 2004,
 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014, 1992, 1997, 1993, 1994,
 1982, 2016, 2003, 1986, 2000,    0, 1995, 1991, 1981, 1987, 1980, 1983]
Length: 38, dtype: Int64

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

array(['E', 'без рейтинга', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'],
      dtype=object)

Выводы о дубликатах.
- В начале мы вывели количество уникальных значений в каждом столбце датафрейма. Как мы помним из первого пункта анализа, всего строк 16956. Можно сделать вывод, что дубликаты есть в каждом столбце. Меньше всего их в столбце name, там более 10 тыс. оригинальных строк. 
Проанализируем каждый столбец: 
- `name`: неявные дубликаты сложно проанализировать из-за большого объема уникальынх строк. Далее попробуем определить явные дубликаты.  
- `platform`:  явных и неявных дубликатов нет. Платформы могут повторятся. Также мы вывели все уникальные значения для проверки. 
- `year_of_release`: года могут повторятся. Также мы вывели все уникальные значения для проверки. 
- `genre`: в этом столбце дубликаты придется исправить. Мы вывели далее все уникальыне значения и можно заметить, что жанры по названию совпадают, но различаются лишь регистром. Это неявные дубликаты. 
- `na_sales`: значения могут повторятся, так как это чисенные значения количества продаж в регионе.             
- `eu_sales`: значения могут повторятся, так как это чисенные значения количества продаж в регионе.           
- `jp_sales`: значения могут повторятся, так как это чисенные значения количества продаж в регионе.           
- `other_sales`: значения могут повторятся, так как это чисенные значения количества продаж.         
- `critic_score`: значения могут повторятся, так как это численные оценки в определенном диапазоне.          
- `user_score`: значения могут повторятся, так как это численные оценки в определенном диапазоне.        
- `rating`: есть неявные дубликаты: это значения K-A и Е. К-А использоваля до 1998 года и после был замененм на категорию Е. Поэтому дальше мы заменим ее на Е для более удобного анализа.
 

- Избавимся от неявных дубликатов в столбце `genre`. Приведем все значения в строке к нижнему регистру.

In [15]:
games['genre'] = games['genre'].str.lower()
display(games['genre'].unique(), games['genre'].nunique())

array(['sports', 'platform', 'racing', 'role-playing', 'puzzle', 'misc',
       'shooter', 'simulation', 'action', 'fighting', 'adventure',
       'strategy'], dtype=object)

12

 - Избавимся от дубликатов в столбце `rating`.


In [16]:
games['rating'] = games['rating'].str.replace('K-A','E')
display(games['rating'].unique())

array(['E', 'без рейтинга', 'M', 'T', 'E10+', 'AO', 'EC', 'RP'],
      dtype=object)

- Можно заметить, что количество уникальных строк уменьшилось в два раза. 

- Попробуем найти явные дубликаты по всем столбцам датафрейма. 

In [17]:
display(games.duplicated(subset = None, keep = 'first'))

0        False
1        False
2        False
3        False
4        False
         ...  
16951    False
16952    False
16953    False
16954    False
16955    False
Length: 16944, dtype: bool

- Можно заметить, что есть дубликаты. Поэтому далее удалим явные дубликаты из столбца. 

In [18]:
games1 = games.drop_duplicates(subset = None, keep = 'first')
display(games1.duplicated(subset = None, keep = 'first'))

0        False
1        False
2        False
3        False
4        False
         ...  
16951    False
16952    False
16953    False
16954    False
16955    False
Length: 16703, dtype: bool

Промежуточный вывод:
- Информацию насчет количества уникальных строк в каждом столбце, явных и неявных дубликатов можно посмотреть выше. 
- После общего анализа дубликатов были преобразованы данные в столбце `genre`. Там имелись невные дубликаты из-за разного регистра. Все данные были приведены к нижнему регистру, что уменьшило количество уникальных значений в два раза: c 24 до 12.
- Также мы преобразовали неявные дубликаты в столбце `rating`: значения рейтинга K-A заменены на Е.
- Далее мы исследовали столбец `name` и обнаружили там явные дубликаты. Уникальных значений в строке всего 11 556, а через метод `.duplicated()` было обнаружено 16944 строки. Сделав выовд о том, что игра с одни и тем же названием не могла выйти несколько раз, было принято решение удалить все явные дубликаты. В итоге мы снова получили 11556 строк. 


In [19]:
# Абсолютное и относительное значение количества пропусков можно посмотреть выше

#Так как удяляли дубликаты мы только из столбца name, поэтому и считаем значения только для него
g = games.shape[0]
g1 = games1.shape[0]
count1 = g-g1 # Абсолютное значение дубликатов
count2 = g/g1 # Относительное значение дубликатов
display(count1, round(count2,2))

# Здесь мы производим такое же удаление дубликатов, что делали выше в переменной games1, только уже с 
# исходным датафреймов, чтобы проводить дальнейший было проще
games = games.drop_duplicates(subset = None, keep = 'first') 
display(games.duplicated(subset = None, keep = 'first'))

241

1.01

0        False
1        False
2        False
3        False
4        False
         ...  
16951    False
16952    False
16953    False
16954    False
16955    False
Length: 16703, dtype: bool

Общий промежуточный вывод:
- Были получены [общие сведения](#1) о датафрейме о количестве строк и столбцоы (16956х11), а также типов данных. 
- Все столбцы приведены к [стилю snake case](#2A). 
- Были преобразованы [типы данных в столбцах](#2B) на более оптимальные. 
- Были посчитаны [все пропуски](#2C) в абсолютных и относительных значениях, выявлены их причины. Были либо заменены значения в пропусках, либо удалены, либо оставлены такими, какие они есть. 
- Были определены [количество уникальных строк по столбцам](#2D), выявлены неявные дубликаты в столбце `genre`. Уникальные строки после преобразваония уменьшились в два раза (с 24 до 12). Также были выявлены и удалены явные дубликаты, а также посчитаны абсолютные и относительные значения дубликатов. 

Более подробно о каждом этапе работы можно прочитать выше, везде были составлены отчеты и объяснения про проделанной работе.

---

## 3. Фильтрация данных
Отберем данные за период с 2000 по 2013 год включительно. Сохраним новый срез данных в отдельном датафрейме `df_actual`. <a id='3'></a>

In [20]:
df_actual = games[(games['year_of_release'] >= 2000) & (games['year_of_release'] <= 2013)]
df_actual = pd.DataFrame(df_actual)
display(df_actual)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E
...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,GC,2003,shooter,0.01,0.00,0.00,0.00,-1.0,-1.0,T
16949,Woody Woodpecker in Crazy Castle 5,GBA,2002,platform,0.01,0.00,0.00,0.00,-1.0,-1.0,без рейтинга
16950,SCORE International Baja 1000: The Official Game,PS2,2008,racing,0.00,0.00,0.00,0.00,-1.0,-1.0,без рейтинга
16952,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,-1.0,-1.0,без рейтинга


---

## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделим все игры по оценкам пользователей и выделим такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала). <a id='4'></a>

In [21]:
df_actual['user_score_category'] =pd.cut(df_actual['user_score'], bins = [-1,0,3,8,10], right = False, labels = ['без оценки','низкая оценка','средняя оценка','высокая оценка'])
display(df_actual)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка
...,...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,GC,2003,shooter,0.01,0.00,0.00,0.00,-1.0,-1.0,T,без оценки
16949,Woody Woodpecker in Crazy Castle 5,GBA,2002,platform,0.01,0.00,0.00,0.00,-1.0,-1.0,без рейтинга,без оценки
16950,SCORE International Baja 1000: The Official Game,PS2,2008,racing,0.00,0.00,0.00,0.00,-1.0,-1.0,без рейтинга,без оценки
16952,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,-1.0,-1.0,без рейтинга,без оценки


- Также была добавлена категория `без оценки` для строчек с пропусками (пропуски заменены на значение -1). 



- Разделим все игры по оценкам критиков и выделим такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [22]:
df_actual['critic_score_category'] =pd.cut(df_actual['critic_score'], bins = [-1,0,30,80,100], right = False, labels = ['без оценки','низкая оценка','средняя оценка','высокая оценка'])
display(df_actual)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,GC,2003,shooter,0.01,0.00,0.00,0.00,-1.0,-1.0,T,без оценки,без оценки
16949,Woody Woodpecker in Crazy Castle 5,GBA,2002,platform,0.01,0.00,0.00,0.00,-1.0,-1.0,без рейтинга,без оценки,без оценки
16950,SCORE International Baja 1000: The Official Game,PS2,2008,racing,0.00,0.00,0.00,0.00,-1.0,-1.0,без рейтинга,без оценки,без оценки
16952,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,-1.0,-1.0,без рейтинга,без оценки,без оценки


- Также была добавлена категория `без оценки` для строчек с пропусками (пропуски заменены на значение -1).

- Cгруппируем данные по выделенным категориям и посчитайем количество игр в каждой категории.

In [23]:
display (df_actual.groupby('user_score_category')['name'].count(), df_actual.groupby('critic_score_category')['name'].count())

/var/folders/xl/_7t_lfxs2cn13d3n1jpw_wgm0000gn/T/ipykernel_2275/1787519838.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display (df_actual.groupby('user_score_category')['name'].count(), df_actual.groupby('critic_score_category')['name'].count())


user_score_category
без оценки        6296
низкая оценка      116
средняя оценка    4078
высокая оценка    2282
Name: name, dtype: int64

critic_score_category
без оценки        5610
низкая оценка       55
средняя оценка    5421
высокая оценка    1686
Name: name, dtype: int64

- Можно заметить, что в обоих категориях больше всего игр `без оценок` (около 4000). Следующие по численности категории: у `user_score_category` это `средняя оценка` (2269); у `critic_score_category` это тоже `средняя оценка` (3280). Больше высоких оценок (1573) ставят пользователи, а не критики (1009). 

- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [24]:
group_values = df_actual.groupby('platform')['name'].count()
group_values = pd.Series(group_values)
sorted_values = group_values.sort_values(ascending = False)
display(sorted_values.head(7))

platform
PS2     2126
DS      2117
Wii     1275
PSP     1179
X360    1118
PS3     1087
GBA      810
Name: name, dtype: int64

- Лидерует среди топ-7 платформ `PS2`: для этой платформы было выпущено 1835 игр. 

---

## 5. Итоговый вывод

Выведем общую информацию об исходном датафрейме `games` и o новом датафрейме `df_actual`, который был создан как срез данных по исходному датафрейму за 2000 и 2013 года. Здесь можно посмотреть, какие столбцы появились, как изменилось количество строк. Описание двух новых столбцов датафрейма `df_actual` `user_score_category` и `critic_score_category` можно прочитать ниже, в общем выводе.

In [25]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16703 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16703 non-null  object 
 1   platform         16703 non-null  object 
 2   year_of_release  16703 non-null  Int64  
 3   genre            16703 non-null  object 
 4   na_sales         16703 non-null  float64
 5   eu_sales         16703 non-null  float64
 6   jp_sales         16703 non-null  float64
 7   other_sales      16703 non-null  float64
 8   critic_score     16703 non-null  float64
 9   user_score       16703 non-null  float64
 10  rating           16703 non-null  object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.5+ MB


In [26]:
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12772 entries, 0 to 16954
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   name                   12772 non-null  object  
 1   platform               12772 non-null  object  
 2   year_of_release        12772 non-null  Int64   
 3   genre                  12772 non-null  object  
 4   na_sales               12772 non-null  float64 
 5   eu_sales               12772 non-null  float64 
 6   jp_sales               12772 non-null  float64 
 7   other_sales            12772 non-null  float64 
 8   critic_score           12772 non-null  float64 
 9   user_score             12772 non-null  float64 
 10  rating                 12772 non-null  object  
 11  user_score_category    12772 non-null  category
 12  critic_score_category  12772 non-null  category
dtypes: Int64(1), category(2), float64(6), object(4)
memory usage: 1.2+ MB


Какие преобразования были сделаны с датафреймом `games`:

- Были получены [общие сведения о датафрейме](#1) о количестве строк и столбцоы (16956х11), а также типов данных.Были выявлены пропуски в таких столбцах: `Name`; `Year of Releaese`; `Genre`; `Critic Score`; `User Score`; `Rating`. Соотвественно, столбцы без пропусков: `Platform`; `NA sales`; `EU sales`; `JP sales`; `Other sales`. Были выявлены столбцы с типом данных `object`, `float64`, нет столбцов с типом данных `integer` и `bool`. Также можно было заметить, что столбцы `NA sales`; `EU sales`; `JP sales`; `Other sales`, описывающие одинаковую информацию в одинаковых единицах измерения имеют разные типы данных (`NA sales`, `Other sales` - `float64`, `EU sales`; `JP sales` - `object`). User Score имел тип данных object.`Year of Releaese` имел тип данных `float64`.
- Привели названия столбцов к стилю [snake case](#2A).
- [Заменили все пропуски](#2B) в числовых колонках на `NaN`. Также для удобства преобразовали колонку `year_of_release` в формат `Int64`. 
- Изучили в каких столбцах [встречаются пропуски](#2C). Были посчитаны относительные и абсолютные значения пропусков в столбцах, пропуски были заменены индикаторыми либо удалены. Также были определены причины, по которым пропуски могли втсретиться в тех или иных столбцах. 
- Изучили вопрос, связанный с [явными и неявными дубликатами](#2D). Были определены количество уникальных строк по столбцам, выявлены неявные дубликаты в столбце `genre`. Все строки по этому столбцу были приведены к нижнему регистру. Уникальные строки после преобразваония уменьшились в два раза (с 24 до 12). Были выявлены и удалены явные дубликаты, а также посчитаны абсолютные и относительные значения дубликатов. 
- Далее провели [фильтрацию данных](#3) и создали новый датафрейм `df_actual` Мы отобрали данные по столбцу `year_of__releaese`. 

Какие преобразования были сделаны с датафреймом `games`:

- Была проведена [категоризация данных](#4). Разделили все игры по оценкам пользователей и выделили такие категории: `высокая оценка` (от 8 до 10 включительно), `средняя оценка` (от 3 до 8, не включая правую границу интервала) и `низкая оценка` (от 0 до 3, не включая правую границу интервала). Столбец с такой категорией получил название `user_score_category`. В данной столбце также появилась категория `без оценки` для значений -1 (индикатор, которым мы заменили пропуск). Разделили все игры по оценкам критиков и выделили такие категории: `высокая оценка` (от 80 до 100 включительно), `средняя оценка` (от 30 до 80, не включая правую границу интервала) и `низкая оценка` (от 0 до 30, не включая правую границу интервала). Столбец с такой категорией получил название `critic_score_category`. В данной столбце также появилась категория `без оценки` для значений -1 (индикатор, которым мы заменили пропуск). Можно заметить, что в обоих категориях больше всего игр `без оценок` (около 4000). Следующие по численности категории: у `user_score_category` это `средняя оценка` (2269); у `critic_score_category` это тоже `средняя оценка` (3280). Больше высоких оценок (1573) ставят пользователи, а не критики (1009).
- Далее выделили топ-7 платформ по количеству игр, выпущенных за весь актуальный период (2000-2013). Ими оказались: PS2, DS, Wii, PSP, PS3, GBA, X360.